In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.feature import PCA

In [2]:
spark = SparkSession\
        .builder\
        .appName("PCA")\
        .getOrCreate()

22/11/10 18:02:16 WARN Utils: Your hostname, darian-Laptop resolves to a loopback address: 127.0.1.1; using 10.0.85.1 instead (on interface outline-tun0)
22/11/10 18:02:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/10 18:02:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/10 18:02:17 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: f0:9b:ea:8e:9a:e2:7a:80


In [3]:
archivo = 'GH70_501_filtered_by_seq_kmers_2_spaced_1.csv'
df_spark = spark.read.csv(archivo,header=True,inferSchema=True)

In [4]:
col = df_spark.columns[1:]

In [5]:
vector_assambler = VectorAssembler(inputCols=col, outputCol='features')

In [6]:
df_temp = vector_assambler.transform(df_spark) 

In [7]:
minmax_scaler = MinMaxScaler(inputCol='features', outputCol='scaled') 

In [8]:
df_trans = minmax_scaler.fit(df_temp).transform(df_temp)

22/11/10 18:02:24 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
pca = PCA(k=50, inputCol="scaled", outputCol="pca_features")
model = pca.fit(df_trans)

22/11/10 18:02:28 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/11/10 18:02:28 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [10]:
p = model.explainedVariance.toArray()
p.sum()
    

0.9066425028574921

In [11]:
spark.stop()